# 3.1 Simulated Annealing

Install required packages / libraries (or update).

In [4]:
pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'


  ERROR: Command errored out with exit status 2:
   command: 'c:\Users\moham\AppData\Local\Programs\Python\Python37-32\python.exe' 'c:\Users\moham\AppData\Local\Programs\Python\Python37-32\lib\site-packages\pip' install --ignore-installed --no-user --prefix 'C:\Users\moham\AppData\Local\Temp\pip-build-env-w0v0c664\overlay' --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple --extra-index-url https://pypi.ngc.nvidia.com --trusted-host pypi.ngc.nvidia.com -- 'cython~=3.0.2' oldest-supported-numpy 'setuptools>=67.8' wheel
       cwd: None
  Complete output (28 lines):
  Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com, https://pypi.ngc.nvidia.com
  ERROR: Exception:
  Traceback (most recent call last):
    File "c:\Users\moham\AppData\Local\Programs\Python\Python37-32\lib\site-packages\pip\_internal\cli\base_command.py", line 188, in _main
    File "c:\Users\moham\AppData\Local\Programs\Python\Python37-32\lib\site-packages\p

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Attempting uninstall: pip
    Found existing installation: pip 20.1.1
    Uninstalling pip-20.1.1:
      Successfully uninstalled pip-20.1.1
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Install Wheels and Pipwin beforehand.

In [ ]:
pip install wheels
pip install pipwin

Use Pipwin to install GDAL.

In [ ]:
pipwin install gdal
pipwin install fiona

In [ ]:
pip install -r requirements.txt

## 3.1.1 Initialization

Begin the program by loading the dataset.

In [ ]:
import pandas as pd

# Load the CSV files
gas_stations_df = pd.read_csv('dataset/gas_stations.csv')
distances_df = pd.read_csv('dataset/x_y_distances.csv')

# Preview the data
gas_stations_df.head(), distances_df.head()

### 3.1.2 Import Necessary Libraries

This probabilistic technique is used for approximating the global optimum of a given function. It will find the shortest route across the gas stations, beginning from DISPERINDAG Surabaya.

In [ ]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt

### 3.1.3 Execution

This section defines the distances, starting point, base temperature, and cooling schedule.

In [ ]:
def simulated_annealing(distances, start_idx, temperature=1000, cooling_rate=0.003, min_temp=1):
    num_nodes = len(distances)
    
    # Generate initial solution (random path)
    current_solution = list(range(num_nodes))
    random.shuffle(current_solution)
    current_solution.insert(0, start_idx)  # Start at DISPERINDAG
    
    current_distance = calculate_total_distance(current_solution, distances)
    best_solution = current_solution.copy()
    best_distance = current_distance
    
    # Annealing process
    while temperature > min_temp:
        # Generate new solution by swapping two nodes
        new_solution = current_solution.copy()
        i, j = random.sample(range(1, num_nodes), 2)  # avoid swapping the start node
        new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
        
        new_distance = calculate_total_distance(new_solution, distances)
        
        # Decide whether to accept the new solution
        if accept_solution(current_distance, new_distance, temperature):
            current_solution = new_solution
            current_distance = new_distance
        
        # Update the best solution if found a new best
        if current_distance < best_distance:
            best_solution = current_solution.copy()
            best_distance = current_distance
        
        # Cool the system
        temperature *= 1 - cooling_rate
    
    return best_solution, best_distance


### 3.1.4 Travel Distance

This section will calculate the total distance traveled according to the number and complexity of nodes.

In [ ]:
def calculate_total_distance(solution, distances):
    total_distance = 0
    for i in range(len(solution) - 1):
        total_distance += distances.loc[solution[i], solution[i + 1]]
    return total_distance

### 3.1.5 Choose Best Route

This section will pick the best / optimal solution according to the thermal expansion and cooling process in the region.

In [ ]:
def accept_solution(current_distance, new_distance, temperature):
    if new_distance < current_distance:
        return True
    else:
        # Accept the new solution with a certain probability
        return random.random() < math.exp((current_distance - new_distance) / temperature)

# 3.2 Lovebird Algorithm

## 3.2.1 Define algorithm

This section provides the necessary features for the algorithm through the number of nodes and agents iterating through them.

In [ ]:
def lovebird_algorithm(distances, start_idx, num_agents=10, max_iterations=100):
    num_nodes = len(distances)
    
    # Initialize agents randomly
    agents = [list(range(num_nodes)) for _ in range(num_agents)]
    for agent in agents:
        random.shuffle(agent)
        agent.insert(0, start_idx)  # Start at DISPERINDAG

    best_agent = None
    best_distance = float('inf')

    for iteration in range(max_iterations):
        for agent in agents:
            # Calculate the total distance for this agent's solution
            current_distance = calculate_total_distance(agent, distances)
            
            # Update the best solution if this agent is better
            if current_distance < best_distance:
                best_distance = current_distance
                best_agent = agent
        
            # Move agents toward the best solution
            for i in range(1, num_nodes):
                if random.random() < 0.1:
                    agent[i] = best_agent[i]

    return best_agent, best_distance
